# Домашнее задание №3. Дедлайн - 21 ноября
Основы машинного обучения. К.Шематоров  
Группа ML-13. __Студент - Усцов Артем Алексеевич__

## Part 0. Service function declaration

Connecting all the libraries necessary for work and declaring functions

In [1]:
# Main libraries
import numpy as np
import pandas as pd

# import keras as ks
import keras as ks
import nltk
import re
import codecs


from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from keras.preprocessing.text import Tokenizer

%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# Train dataset
train_df = pd.read_csv("/content/drive/MyDrive/ML_Techno_2021/data/train.csv")
train_df.head()

In [4]:
# Check the empty data
train_df.isnull().any()

In [5]:
# Test dataset
test_df = pd.read_csv("/content/drive/MyDrive/ML_Techno_2021/data/test.csv")
test_df.head()

In [6]:
# Check the empty data
test_df.isnull().any()

Как видно, пропусков в данных не имеется. Дополнительная обработка данного случая не требуется

In [7]:
print(f"Длина вектора данных на обучении - {len(train_df)}")
print(f"Длина вектора данных на тесте - {len(test_df)}")
print(f"Соотношение теста к обучающим - {round(len(test_df) / len(train_df), 2)}")

Имеем перекос в размере данных на обучающей выборке на 22%

In [8]:
# Labels balance
train_df["target"].value_counts()

In [9]:
X_train = train_df["title"].values
X_test = test_df["title"].values
y_train = train_df["target"].astype(int).values

## Part 1. Simple baseline realisation

In [10]:
y_pred = [int("порно" in text) for text in X_train]

In [11]:
print(classification_report(y_train, y_pred, digits=3))

In [12]:
print(f"AUC-ROC metric: {round(roc_auc_score(y_train, y_pred), 3)}")
fpr, tpr, _ = roc_curve(y_train, y_pred)

plt.plot(fpr, tpr, label="Simple baseline case")
plt.legend()
plt.grid()
plt.title('ROC curve')

### Submit

In [13]:
test_df["target"] = [("порно" in text) for text in X_test]

# Create file and read in stdout
test_df[["id", "target"]].to_csv("simple_baseline.csv", index=False)
!cat simple_baseline.csv | head

### Не все так однозначно

**не порно**:
- Болезни опорно-двигательной системы и импотенция: взаимосвязь
- Транссексуальные рыбы - National Geographic Россия: красота мира в каждом кадре
- Групповая обзорная экскурсия по Афинам - цена €50
- Больного раком Задорнова затравили в соцсетях.
- Гомосексуалисты на «Первом канале»? Эрнст и Галкин – скрытая гей-пара российского шоу-бизнеса | Заметки о стиле, моде и жизни

**порно**:
- Отборная домашка
- Сюзанна - карьера горничной / Susanna cameriera perversa (с русским переводом) 1995 г., DVDRip

## Conclusion:

Требуется добавить отслеживание "схожести" слов, а также важен порядок следования слов в предложении

## Part 1. ML baseline realisation
Использование базовой векторизации и простейшей модели классификации - мультиномиального наивного байесовского классификатора

In [14]:
vectorizer = CountVectorizer()
model = MultinomialNB()

In [15]:
X_train_vectorized = vectorizer.fit_transform(X_train)

In [16]:
feature_names = np.array(vectorizer.get_feature_names())

In [17]:
id_ = 42
print(X_train[id_])
x_vector = X_train_vectorized.getrow(id_).toarray()[0]
[feature for feature in feature_names[x_vector > 0]]

In [18]:
%%time

model.fit(X_train_vectorized, y_train)
y_pred = model.predict(X_train_vectorized)

In [19]:
print(classification_report(y_train, y_pred, digits=3))

In [20]:
print(f"AUC-ROC metric: {round(roc_auc_score(y_train, y_pred), 3)}")
fpr, tpr, _ = roc_curve(y_train, y_pred)

plt.plot(fpr, tpr, label="Simple baseline case")
plt.legend()
plt.grid()
plt.title('ROC curve')

### Submit

In [21]:
X_test_vectorized = vectorizer.transform(X_test)
test_df["target"] = model.predict(X_test_vectorized).astype(bool)

# Create file and read in stdout
test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)
!cat ml_baseline.csv | head

## Conclusion:

Как видно, токенизация слов и простейший классификатор хоть и дают результаты лучше, чем при простом отсеивании по ключевому слову, однако качество все равно оставляет желать лучшего. Попробуем увеличить качество

## Part 2. Smart data processing

In [22]:
train_df.info()

In [23]:
# Имеется большой перекос в сторону классификации текста как не порносодержащего
train_df.groupby("target").count()

### Sanitizing input
Let's make sure our tweets only have characters we want. We remove '#' characters but keep the words after the '#' sign because they might be relevant (eg: #disaster)

In [ ]:
input_file = codecs.open("/content/drive/MyDrive/ML_Techno_2021/data/train.csv", "r", encoding='utf-8', errors='replace')
output_file = open("/content/drive/MyDrive/ML_Techno_2021/data/train.csv", "w")

def sanitize_characters(raw, clean):    
    for line in input_file:
        out = line
        output_file.write(line)

sanitize_characters(input_file, output_file)

In [ ]:
# Train dataset
train_df = pd.read_csv("/content/drive/MyDrive/ML_Techno_2021/data/train.csv/train_clean.csv")
train_df.head(10)

Уберем лишние символы в строчках - все возможные запятые, служебные символы и так далее. Также произведем приведение слов к нормальной форме и к нижнему регистру

In [ ]:
import pymorphy2
import re

ma = pymorphy2.MorphAnalyzer()

def clean_text(text, encoding=False):
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text) #deleting symbols  
    text = " ".join(ma.parse(word)[0].normal_form for word in text.split())
    text = ' '.join(word for word in text.split() if len(word)>3)
    if encoding:
        text = text.encode("utf-8")

    return text

In [ ]:
# It's too long. DO NOT RUN

# train_df['title_clean'] = train_df.apply(lambda x: clean_text(x[u'title']), axis=1)
# train_df['title_clean'] = train_df['title_clean'].astype("str")
# test_df['title_clean'] = test_df.apply(lambda x: clean_text(x[u'title']), axis=1)
# test_df['title_clean'] = test_df['title_clean'].astype("str")

In [ ]:
# train_df.to_csv("clean_train_df.csv")
# test_df.to_csv("clean_test_df.csv")

In [ ]:
# It's too long. DO NOT RUN

# train_df['title_clean'] = train_df.apply(lambda x: clean_text(x[u'title'], True), axis=1)
# train_df['title_clean'] = train_df['title_clean'].astype("str")
# test_df['title_clean'] = test_df.apply(lambda x: clean_text(x[u'title'], True), axis=1)
# test_df['title_clean'] = test_df['title_clean'].astype("str")

In [ ]:
# # Будем использовать уже готовые датасеты, так как онлайн обработка производится существенное время
clean_train_df = pd.read_csv("/content/drive/MyDrive/ML_Techno_2021/data/train.csv/clean_train_df.csv")
clean_train_df['title_clean'] = clean_train_df['title_clean'].astype("str")
clean_test_df = pd.read_csv("/content/drive/MyDrive/ML_Techno_2021/data/train.csv/clean_test_df.csv")
clean_test_df['title_clean'] = clean_test_df['title_clean'].astype("str")

In [ ]:
clean_train_df.head(50)

In [ ]:
# Как можно видеть, после нормализации и очистки, мы имеем строчки со значением nan - уберем их из датасета
clean_train_df = clean_train_df[clean_train_df['title_clean'] != 'nan']

In [ ]:
# На первом этапе произведем токенизацию слов
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
clean_train_df["tokens"] = clean_train_df["title_clean"].apply(tokenizer.tokenize)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

all_words = [word for tokens in clean_train_df["tokens"] for word in tokens]
sentence_lengths = [len(tokens) for tokens in clean_train_df["tokens"]]

VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max sentence length is %s" % max(sentence_lengths))

In [ ]:
# Посмотрим на распределение длин слов
fig = plt.figure(figsize=(10, 10)) 
plt.xlabel('Sentence length')
plt.ylabel('Number of sentences')
plt.hist(sentence_lengths)
plt.show()

## Part 2.1 Tokenizing
Используем токенизацию слов на чистом датасете

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def cv(data):
    count_vectorizer = CountVectorizer()
    emb = count_vectorizer.fit_transform(data)
    return emb, count_vectorizer

list_corpus = clean_train_df["title_clean"].tolist()
list_labels = clean_train_df["target"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.30, 
                                                                              random_state=40)

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)

Попробуем спроецировать все наше n-мерное пространство признаков в 2-х мерное пространство при помощи метода главных компонент, чтобы проверить, насколько хорошо разделимы наши данные

In [ ]:
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib
import matplotlib.patches as mpatches


def plot_LSA(test_data, test_labels, savepath="/content/drive/MyDrive/ML_Techno_2021/data/train.csv/PCA_demo.csv", plot=True):
        lsa = TruncatedSVD(n_components=2)
        lsa.fit(test_data)
        lsa_scores = lsa.transform(test_data)
        color_mapper = {label:idx for idx,label in enumerate(set(test_labels))}
        color_column = [color_mapper[label] for label in test_labels]
        colors = ['orange','blue','blue']
        if plot:
            plt.scatter(lsa_scores[:,0], lsa_scores[:,1], s=8, alpha=.8, c=test_labels, cmap=matplotlib.colors.ListedColormap(colors))
            red_patch = mpatches.Patch(color='orange', label='Irrelevant')
            green_patch = mpatches.Patch(color='blue', label='Porn')
            plt.legend(handles=[red_patch, green_patch], prop={'size': 30})


fig = plt.figure(figsize=(16, 16))          
plot_LSA(X_train_counts, y_train)
plt.show()

Под меткой "Irrelevant" скрывается все, что не относитеся к порнографии

Обучим модель логистической регресссии и посмотрим на ее качество.
Все составляющие модели были ранее подобраны при помощи перебора по сетке. Данный этап здесь опущен, так как занимал достаточно большое время

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf.fit(X_train_counts, y_train)
y_predicted_counts = clf.predict(X_test_counts)

In [ ]:
print(classification_report(y_test, y_predicted_counts, digits=3))

In [ ]:
print(f"AUC-ROC metric: {round(roc_auc_score(y_test, y_predicted_counts), 3)}")
fpr, tpr, _ = roc_curve(y_test, y_predicted_counts)

plt.plot(fpr, tpr, label="Simple baseline case")
plt.legend()
plt.grid()
plt.title('ROC curve')

Качество заметно улучшилось по сравнению с обычной моделью. Однако все равно есть маневры для улучшения.
Посмотрим на то, где наша модель ошибается при помощи confusion matrix

In [ ]:
import numpy as np
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.winter):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize=20)
    plt.yticks(tick_marks, classes, fontsize=20)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", 
                 color="white" if cm[i, j] < thresh else "black", fontsize=40)
    
    plt.tight_layout()
    plt.ylabel('True label', fontsize=30)
    plt.xlabel('Predicted label', fontsize=30)

    return plt

In [ ]:
cm = confusion_matrix(y_test, y_predicted_counts)
fig = plt.figure(figsize=(10, 10))
plot = plot_confusion_matrix(cm, classes=['Irrelevant','Porn'], normalize=False, title='Confusion matrix')
plt.show()
print(cm)

Наша модель склонна к ошибке первого рода - детектирует "порно" как "не порно". Что с точки зрения пользователя достаточно критично. Посмотрим глубже, как наша модель воспринимает слова и в какую категорию их относит

In [ ]:
def get_most_important_features(vectorizer, model, n=5):
    index_to_word = {v:k for k,v in vectorizer.vocabulary_.items()}

    # loop for each class
    classes ={}
    for class_index in range(model.coef_.shape[0]):
        word_importances = [(el, index_to_word[i]) for i,el in enumerate(model.coef_[class_index])]
        sorted_coeff = sorted(word_importances, key = lambda x : x[0], reverse=True)
        tops = sorted(sorted_coeff[:n], key = lambda x : x[0])
        bottom = sorted_coeff[-n:]

        classes[class_index] = { 'tops':tops, 'bottom':bottom }

    return classes

In [ ]:
importance = get_most_important_features(count_vectorizer, clf, 20)

In [ ]:
def plot_important_words(top_scores, top_words, bottom_scores, bottom_words, name):
    y_pos = np.arange(len(top_words))
    top_pairs = [(a,b) for a,b in zip(top_words, top_scores)]
    top_pairs = sorted(top_pairs, key=lambda x: x[1])
    
    bottom_pairs = [(a,b) for a,b in zip(bottom_words, bottom_scores)]
    bottom_pairs = sorted(bottom_pairs, key=lambda x: x[1], reverse=True)
    
    top_words = [a[0] for a in top_pairs]
    top_scores = [a[1] for a in top_pairs]
    
    bottom_words = [a[0] for a in bottom_pairs]
    bottom_scores = [a[1] for a in bottom_pairs]
    
    fig = plt.figure(figsize=(10, 10))  

    plt.subplot(121)
    plt.barh(y_pos,bottom_scores, align='center', alpha=0.5)
    plt.title('Irrelevant', fontsize=20)
    plt.yticks(y_pos, bottom_words, fontsize=14)
    plt.suptitle('Key words', fontsize=16)
    plt.xlabel('Importance', fontsize=20)
    
    plt.subplot(122)
    plt.barh(y_pos,top_scores, align='center', alpha=0.5)
    plt.title('Porn', fontsize=20)
    plt.yticks(y_pos, top_words, fontsize=14)
    plt.suptitle(name, fontsize=16)
    plt.xlabel('Importance', fontsize=20)
    
    plt.subplots_adjust(wspace=0.8)
    plt.show()

In [ ]:
top_scores = [a[0] for a in importance[0]['tops']]
top_words = [a[1] for a in importance[0]['tops']]
bottom_scores = [a[0] for a in importance[0]['bottom']]
bottom_words = [a[1] for a in importance[0]['bottom']]

plot_important_words(top_scores, top_words, bottom_scores, bottom_words, "Most important words for relevance")

В целом, модель справляется достаточно хорошо.
Однако есть и промахи - "свинг" явно относится к категории "порно", а "пора" наоборот.  
Требуется улучшение

## Part 2.2. TFIDF
Попробуем добавить важность конкретного слова в конкретном описании при помощи tfidf

In [ ]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()
    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
fig = plt.figure(figsize=(16, 16))          
plot_LSA(X_train_tfidf, y_train)
plt.show()

Как видно, разделимость классов стала много лучше, однако все равно один класс как бы "лежит" внутри другого и для моделей "классической" линейной классификации трудно будет их различить

In [ ]:
param_grid = {"C": [1, 5, 10, 30, 40, 50], "class_weight" : ["balanced"], 
             "solver": ["newton-cg"], "multi_class" : ["multinomial"]}

In [ ]:
estimator = LogisticRegression(n_jobs=-1, random_state=40)
clf_tfidf = GridSearchCV(estimator, param_grid, cv = 5)
clf_tfidf.fit(X_train_tfidf, y_train)

In [ ]:
clf_tfidf.best_estimator_

Обучим два классификатора - логистическую регрессию и SGD, сравним их качество.
Для логистической регрессии параметры были получены ранее при помощи перебора по сетке.

In [ ]:
clf_tfidf = LogisticRegression(C=10.0, class_weight='balanced', solver='newton-cg', 
                               multi_class='multinomial', n_jobs=-1, random_state=40)

clf_tfidf.fit(X_train_tfidf, y_train)
y_predicted_tfidf = clf_tfidf.predict(X_test_tfidf)

In [ ]:
print(classification_report(y_test, y_predicted_tfidf, digits=4))

TFIDF позволил выиграть в качестве модели, но не слишком много

In [ ]:
param_grid = {"alpha": [0.0001, 0.001, 0.01], "class_weight" : ["balanced", None], "eta0" : [0.1, 0.2, 0.5]}

In [ ]:
SGDClassifier?

In [ ]:
estimator = SGDClassifier(n_jobs=-1, random_state=40, learning_rate = 'adaptive', loss = 'perceptron', early_stopping = True, 
                          validation_fraction = 0.2, eta0=0.1)
clf_tfidf = GridSearchCV(estimator, param_grid, cv = 5)
clf_tfidf.fit(X_train_tfidf, y_train)

In [ ]:
clf_tfidf.best_estimator_

In [ ]:
clf_tfidf = SGDClassifier(random_state=40,
                          learning_rate = 'adaptive', eta0 = 0.1,
                          loss = 'perceptron', alpha=0.0001,
                          early_stopping = True, validation_fraction = 0.2,
                          n_jobs = -1)
clf_tfidf.fit(X_train_tfidf, y_train)

y_predicted_tfidf_sgd = clf_tfidf.predict(X_test_tfidf)

In [ ]:
print(classification_report(y_test, y_predicted_tfidf_sgd, digits=4))

SGD классификатор только ухудшил качество модели

In [ ]:
print(f"AUC-ROC metric: {round(roc_auc_score(y_test, y_predicted_tfidf), 3)}")
fpr, tpr, _ = roc_curve(y_test, y_predicted_tfidf)

plt.plot(fpr, tpr, label="Simple baseline case")
plt.legend()
plt.grid()
plt.title('ROC curve')

In [ ]:
cm2 = confusion_matrix(y_test, y_predicted_tfidf)
fig = plt.figure(figsize=(10, 10))
plot = plot_confusion_matrix(cm2, classes=['Irrelevant','Disaster'], normalize=False, title='Confusion matrix')
plt.show()
print("TFIDF confusion matrix")
print(cm2)
print("BoW confusion matrix")
print(cm)

Модель стала меньше ошибаться в ошибках I рода.
Посмотрим глубже, как наша модель воспринимает слова и в какую категорию их относит

In [ ]:
importance_tfidf = get_most_important_features(tfidf_vectorizer, clf_tfidf, 20)

In [ ]:
top_scores = [a[0] for a in importance_tfidf[0]['tops']]
top_words = [a[1] for a in importance_tfidf[0]['tops']]
bottom_scores = [a[0] for a in importance_tfidf[0]['bottom']]
bottom_words = [a[1] for a in importance_tfidf[0]['bottom']]

plot_important_words(top_scores, top_words, bottom_scores, bottom_words, "Most important words for relevance")

Все равно имеются промахи в словах

In [ ]:
X_test_main = test_df["title"].tolist()
X_test_tfidf = tfidf_vectorizer.transform(X_test_main)

test_df["target"] =  clf_tfidf.predict(X_test_tfidf)

In [ ]:
# Create file and read in stdout

test_df[["id", "target"]].to_csv("ml_tfidf.csv", index=False)
!cat ml_tfidf.csv | head

# Conclusion:
На Kaggle данная модель показала 0.95466 по f1-метке.